
Smart Budget Analyzer & Fraud Detection System

Description: This system is a menu-driven Python application that tracks financial transactions (Income/Expense). It allows users to view spending by category, checks for valid data entry, and uses advanced logic to detect fraud (high-value or repeated transactions).

Python Concepts Used: Classes, Dictionaries, Sets, Tuples, List Comprehensions, Loops, and Tuple Unpacking.



In [3]:
BUDGET_START = 5000
THRESHOLD = 500

my_Category = {"Food", "Rent", "Transport", "Entertainment", "Health", "Other"}


In [4]:

class Transaction:
    def __init__(self, date, amount, category, ttype, description):
        self.date = date
        self.amount = amount
        self.category = category
        self.type = ttype
        self.description = description

    def to_tuple(self):
        return (self.date, self.amount, self.category, self.type, self.description)

    def display(self):
        print(self.to_tuple())


class BudgetTracker:
    def __init__(self):
        self.transactions = []
        self.suspicious = []

    def addTransaction(self, tr):
        self.transactions.append(tr)
        if tr.type == "expense" and tr.amount > THRESHOLD:
            self.suspicious.append(tr)

    def DiplayAll(self):
        for tr in self.transactions:
            tr.display()

    def Displaysuspicious(self):
        for tr in self.suspicious:
            tr.display()

    def CalculateTotal(self):
        totalExpenses = 0
        totalIncome = 0
        for tr in self.transactions:
            if tr.type == "expense":
                totalExpenses += tr.amount
            elif tr.type == "income":
                totalIncome += tr.amount
        return totalExpenses, totalIncome

    def Current_Balance(self):
        totalExpenses, totalIncome = self.CalculateTotal()
        return BUDGET_START + totalIncome - totalExpenses

    def SpendingbyCategory(self):
        category_spending = {cat: 0 for cat in my_Category}
        for tr in self.transactions:
            if tr.type == "expense":
                category_spending[tr.category] += tr.amount
        return category_spending

    def UniqueCategoriesUsed(self):
        return {tr.category for tr in self.transactions}

    def monthly_summary(self, month, year):
        income = 0
        expense = 0
        for tr in self.transactions:
            d, m, y = tr.date.split("-")
            if m == month and y == year:
                if tr.type == "income":
                    income += tr.amount
                else:
                    expense += tr.amount
        return income, expense



In [5]:

def ValidationFun(tr, tracker):
    if tr.amount <= 0:
        return False
    if tr.category not in my_Category:
        return False
    if tr.type not in ("income", "expense"):
        return False
    if tr.type == "expense" and tr.amount > tracker.Current_Balance():
        return False
    return True


def CreateTransaction(date, amount, category, ttype, description, tracker):
    tr = Transaction(date, amount, category, ttype, description)
    if not ValidationFun(tr, tracker):
        return None, False
    tracker.addTransaction(tr)
    is_large = tr.type == "expense" and tr.amount > THRESHOLD
    return tr, is_large

In [6]:


def high_value_expenses(tracker):
    return [tr for tr in tracker.transactions if tr.type == "expense" and tr.amount > THRESHOLD]


def transactions_as_tuples(tracker):
    return [tr.to_tuple() for tr in tracker.transactions]


def fraud_detection(tracker):
    suspicious_large = []
    same_day_category = {}
    for tr in tracker.transactions:
        if tr.type == "expense" and tr.amount > THRESHOLD:
            suspicious_large.append(tr)
        if tr.type == "expense":
            key = (tr.date, tr.category)
            same_day_category[key] = same_day_category.get(key, 0) + 1
    repeated = [key for key, count in same_day_category.items() if count > 1]
    return suspicious_large, repeated

In [7]:
def preload_data(tracker):
    print("Initializing system with test data...")
    CreateTransaction("01-01-2024", 5000, "Other", "income", "Salary", tracker)
    
    CreateTransaction("02-01-2024", 50, "Food", "expense", "Lunch", tracker)
    CreateTransaction("03-01-2024", 100, "Transport", "expense", "Gas", tracker)
    CreateTransaction("04-01-2024", 150, "Health", "expense", "Checkup", tracker)
    CreateTransaction("05-01-2024", 200, "Food", "expense", "Groceries", tracker)
    
    CreateTransaction("10-01-2024", 600, "Rent", "expense", "Late Rent Fee", tracker)
    
    CreateTransaction("12-01-2024", 20, "Entertainment", "expense", "Movie Ticket", tracker)
    CreateTransaction("12-01-2024", 20, "Entertainment", "expense", "Popcorn", tracker)
    
    CreateTransaction("15-01-2024", 30, "Food", "expense", "Coffee", tracker)
    CreateTransaction("16-01-2024", 40, "Transport", "expense", "Uber", tracker)
    
    print("...Data pre-loaded successfully.\n")



In [ ]:

def menu():
    bt = BudgetTracker()
    
    preload_data(bt)
    
    while True:
        print("\n--- BUDGET ANALYZER MENU ---")
        print("1. Add Income")
        print("2. Add Expense")
        print("3. View Transactions")
        print("4. View Balance")
        print("5. Spending by Category")
        print("6. Fraud Detection Report")
        print("7. Monthly Summary")
        print("8. Unique Categories Used") 
        print("9. Exit")
        
        choice = input("Select an option (1-9): ")
        
        if choice == "1" or choice == "2":
            date = input("Date (DD-MM-YYYY): ")
            try:
                amount = float(input("Amount: "))
            except ValueError:
                print("Error: Amount must be a number.")
                continue
                
            print(f"Valid Categories: {my_Category}")
            category = input("Category: ") 
            desc = input("Description: ")
            
            ttype = "income" if choice == "1" else "expense"
            
            tr, is_large = CreateTransaction(date, amount, category, ttype, desc, bt)
            
            if tr:
                print("Transaction Added Successfully.")
                if is_large:
                    print("WARNING: This transaction was flagged as a HIGH VALUE expense.")
            else:
                print("FAILED: Transaction invalid. Check Category, negative amounts, or overdraft.")

        elif choice == "3":
            print("\n--- All Transactions ---")
            bt.DiplayAll()

        elif choice == "4":
            bal = bt.Current_Balance()
            print(f"\nCurrent Balance: ${bal}")

        elif choice == "5":
            print("\n--- Spending by Category ---")
            print(bt.SpendingbyCategory())

        elif choice == "6":
            print("\n--- FRAUD DETECTION REPORT ---")
            large_tx, repeated_tx = fraud_detection(bt)
            
            print(f"Suspicious Large Transactions (> {THRESHOLD}):")
            if not large_tx:
                print("None found.")
            for tr in large_tx:
                tr.display()
                
            print(f"\nRepeated Activity (Same Day & Category):")
            if not repeated_tx:
                print("None found.")
            else:
                print(repeated_tx)

        elif choice == "7":
            m = input("Enter Month (MM): ")
            y = input("Enter Year (YYYY): ")
            inc, exp = bt.monthly_summary(m, y)
            print(f"\nSummary for {m}-{y}:")
            print(f"Total Income: ${inc}")
            print(f"Total Expense: ${exp}")
            print(f"Net Savings: ${inc - exp}")

        elif choice == "8":
            print("\n--- Unique Categories Used ---")
            print(bt.UniqueCategoriesUsed())

        elif choice == "9":
            print("Exiting system. Goodbye!")
            break2
            
        else:
            print("Invalid choice, please try again.")

if __name__ == "__main__":
    menu()

Initializing system with test data...
...Data pre-loaded successfully.


--- BUDGET ANALYZER MENU ---
1. Add Income
2. Add Expense
3. View Transactions
4. View Balance
5. Spending by Category
6. Fraud Detection Report
7. Monthly Summary
8. Unique Categories Used
9. Exit
Valid Categories: {'Rent', 'Other', 'Transport', 'Health', 'Food', 'Entertainment'}
FAILED: Transaction invalid. Check Category, negative amounts, or overdraft.

--- BUDGET ANALYZER MENU ---
1. Add Income
2. Add Expense
3. View Transactions
4. View Balance
5. Spending by Category
6. Fraud Detection Report
7. Monthly Summary
8. Unique Categories Used
9. Exit

Current Balance: $8790

--- BUDGET ANALYZER MENU ---
1. Add Income
2. Add Expense
3. View Transactions
4. View Balance
5. Spending by Category
6. Fraud Detection Report
7. Monthly Summary
8. Unique Categories Used
9. Exit

Current Balance: $8790

--- BUDGET ANALYZER MENU ---
1. Add Income
2. Add Expense
3. View Transactions
4. View Balance
5. Spending by Category
6.

In [9]:
menu()


Initializing system with test data...
...Data pre-loaded successfully.


--- BUDGET ANALYZER MENU ---
1. Add Income
2. Add Expense
3. View Transactions
4. View Balance
5. Spending by Category
6. Fraud Detection Report
7. Monthly Summary
8. Unique Categories Used
9. Exit
Valid Categories: {'Rent', 'Other', 'Transport', 'Health', 'Food', 'Entertainment'}
FAILED: Transaction invalid. Check Category, negative amounts, or overdraft.

--- BUDGET ANALYZER MENU ---
1. Add Income
2. Add Expense
3. View Transactions
4. View Balance
5. Spending by Category
6. Fraud Detection Report
7. Monthly Summary
8. Unique Categories Used
9. Exit
Exiting system. Goodbye!
